# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [2]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [3]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [12]:
browser = webdriver.Chrome(executable_path = '../chromedriver.exe')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

def scroll_to_end(browser):
    last_height = browser.execute_script('return document.body.scrollHeight')
    count = 1
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = browser.execute_script('return document.body.scrollHeight')
        print('第{}次更新高度:'.format(count),new_height)
        count += 1
        if new_height == last_height:
            print('--到達頁面底端--')
            break
        else:
            last_height = new_height
            print('--網頁更新中--')
scroll_to_end(browser)

第1次更新高度: 9066
--網頁更新中--
第2次更新高度: 11502
--網頁更新中--
第3次更新高度: 13804
--網頁更新中--
第4次更新高度: 16209
--網頁更新中--
第5次更新高度: 18520
--網頁更新中--
第6次更新高度: 20804
--網頁更新中--
第7次更新高度: 23132
--網頁更新中--
第8次更新高度: 24530
--網頁更新中--
第9次更新高度: 24580
--網頁更新中--
第10次更新高度: 24580
--到達頁面底端--


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [11]:
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)


### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [ ]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 敏成配合新型冠狀病毒防疫　24小時不停機增產「醫療用口罩濾材」

▲醫療用口罩。（圖／資料畫面）
記者姚惠茹／台北報導
因應近期爆發的新型冠狀病毒疫情，台灣口罩需求大增，敏成（4431）今（6）日宣布配合政府防疫，已經從開工日就增產熔噴技術不織布，也就是醫療用口罩所需的濾材，並將產能火力全開，24小時不停機生產，全力支援防疫所需。
敏成董事長古思明表示，有鑑於春節假期中疫情持續擴大，敏成已經在開工當日就把產能火力全開，24小時不停機加緊趕工，以台灣民眾健康及國家安全為優先，全力配合政府需求生產口罩濾材，為社會盡一份心力。
敏成是成立近30年的熔噴不織布廠商，具有卓越的熔噴技術與穩定性高製程經驗，採用最佳等級的熔噴過濾材原料，以及與世界接軌的全套自主檢驗設備，產製空氣、水等各式濾材與保溫棉等上游材料，符合國際大廠之規範。
敏成表示，目前歐、美及台灣的一線醫材大廠皆是敏成的客戶，現有口罩濾材訂單已排程到五月份，同時敏成也將過濾材料研發不斷提升，已完成可以吸附並殺死細菌的濾材，讓原料量產到位，並可依市場需求提供適當等級的濾材。
敏成指出，近年來社會大眾對於預防空汙、過敏、流感，以及易傳染的病毒日漸重視，因衛生因素使口罩成為日常必需品，並在這次新型冠狀病毒爆發後，醫療型口罩需求大增，因此敏成目前傾全力生產，協助政府佈署防疫任務，期待疫情能早日控制，讓社會大眾能回復到正常生活。
----------------------------------------------------------------------

[1] 市調機構：Apple Watch去年賣出3100萬支、年增36%　超越瑞士鐘錶產業總銷量　

▲Apple watch銷量持續成長。（圖／Ettoday資料庫）
記者謝仁傑／綜合報導
除了銷售量優於瑞士鐘錶產業，Apple Watch銷售較前年增長了36%，銷售量從2250萬支增至3070萬支；反觀瑞士鐘錶產業的銷售量較前年下降約13%，從2420萬支降至2110萬支。
Strategy Analytics高級分析師強調，Apple Watch對年輕消費族群特別有吸引力；而傳統的瑞士製錶商，像是Swat

▲總統蔡英文(圖右)昨（5）日下午視察口罩新產線，經濟部長沈榮津（前排左一）陪同解說，政府斥資2億佈建的 62條口罩產線將陸續上線。（圖／總統府提供）
記者林淑慧／台北報導
武漢肺炎疫情延燒，口罩實名制今（6）日上路，蔡英文總統昨日親赴台灣康匠公司視察口罩新產線，陪同的經濟部長沈榮津表示，為提升產能，經濟部已派員直接駐點在工廠，日夜趕工「全力衝產量」。
依據中央流行疫情指揮中心統計，台灣目前有10名嚴重特殊傳染性肺炎（武漢肺炎）患者，10名個案狀況穩定，且有1人採檢都是陰性，可望近日出院，另9名患者也都是輕症，並赴中港澳旅遊回台者，都要進行居家檢疫 14 天，國內口罩需求大增。
經濟部表示，感謝全國許許多多本土口罩工廠，從年假時開始就配合政府提前開工。像是昨天蔡總統參訪的台灣康匠，陳董事長打趣說，他年節休假時，都在接經濟部的電話，因此年初二工廠就開工生產。
「好幾家（口罩）廠商也都一樣，陸續提早開工，也配合轉為全天候不分日夜，都在生產的模式。」隨行的經濟部官員說，為了促成政府民間一條心，經濟部相關同仁春節沒有休假，數不清的會議、電話跟訪廠，都是為了要全力衝產量，現在，很多人甚至已經直接駐點在工廠。
本月政府斥資2億佈建的 62條口罩產線將陸續上線，國內產能可望逐步提高，以因應國人防疫需求；官員指出，政府對口罩生產的原料亦有掌握，從PP聚丙烯（不織布原料）、到口罩內層ES、中間層熔噴布、外層紡黏，國內都有生產，經濟部將隨時注意原物料的供應情況。
----------------------------------------------------------------------

[15] 毛小孩的健康東森寵物雲包了！　2020開春衝上業界第一品牌

寵物小組／綜合報導
武漢肺炎全球蔓延，人的健康由防疫專家、醫護人員層層把關。東森國際（2614）投資的子公司「東森寵物雲」在這波疫情全心守護毛小孩的健康。在東森寵物雲的專業獲得大家的信任下，2020年開春首月開出紅盤，單月營收創新高！
東森寵物雲董事長兼總經理陳安祥表示，東森寵物雲商店繼去年年底宣布擁有 91 家門市（含奧斯卡連鎖寵物體系），成為全台寵物門市規模第一之後，2020年首月即開紅盤，根據東森寵物雲商店自結損益報告顯示，在經過大幅擴張、整合之後，自2018年成立迄今短短14個月的時間，實現單月獲利，202

▲友達財務長曾煜智。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
面板廠友達（2409）今（5）日晚間召開重大訊息說明會，由財務長曾煜智說明董事會決議通過，公開收購工業電腦大廠凌華（6166），以每股新台幣57元收購凌華普通股5%～30%。以凌華今日收盤價47.05計算，溢價21.1%，以60天平均股價計算，溢價率為20.5%。
曾煜智強調，凌華作為全球工業電腦領導廠商，在工業與商業應用的許多垂直場域，均已展現領先市場的邊緣運算（edge computing）相關的產品技術與解決方案的競爭力，並在既有工業電腦的基礎上，凌華面對客戶對人工智能應用的需求，提出可擴充性的軟硬體平台整合方案。
展望5G時代來臨，曾煜智表示，友達期望借重凌華在工業電腦領域累積多年的經驗，加上友達做為人機介面的優質面板產品，期望雙方締結策略夥伴促成優勢互補，共同掌握多元領域客戶從自動化到智能化的數位轉型需求。
----------------------------------------------------------------------

[30] 台指期開高走低、收盤漲8點　分析師：近期要快節奏操作、疫情過後再做長線

▲台指期2月5日走勢。（圖／取自期交所）
記者余弦妙／台北報導
台指期今（5）早受美股激勵一開盤上漲69點，來到11,603點，接著突然來個大幅度拉回，來到11,474點，翻轉下跌60點，不過探底過後就持續反彈走升，終場以11,542點作收，上漲8點，與現貨相較，逆價差31.62點，讓投資人心情大洗三溫暖。展望後市，群益期貨分析師表示，近期不適合長線布局，適合快節奏的操作，明天仍有機會看漲。
分析師表示，今天探底後就一路向上拉抬，整體盤勢來看比較像是恐慌過後的情緒消散，目前先以1月8日的低點11700點左右當作觀察點，且今天外資還是有小幅度回補的情況來看，目前對於市場行情就抱持中性看法即可。
分析師也說，但根據目前市場消息面來看，已傳出有疫苗可治療武漢肺炎的消息，明天應該還是會呈現持續走升的盤勢。
此外，分析師指出，雖然從長遠來看，經濟數據一定會受到影響，但短期而言目前疫情並無出現重大感染，且美股最近表現亮眼，因此短期內，台指期還是有機會緩步走升，不過近期操作應該要走速戰速決的路線，不宜做長線布局，畢竟疫情尚未完全消失，等疫情消失後再布局才是較好時機。
------